mkdate : 2020-05-03-SUN

# 2강. 다중클래스 케라스 ~ 3강. 다층 퍼셉트론(MLP)

## 강의 :  딥러닝 입문에서 활용까지 케라스(Keras) 

##  제공처 : SKplanet Tacademy

## Youtube

-  [2강  Link](https://www.youtube.com/watch?v=IPR2bYFa6Rw&list=PL9mhQYIlKEheoq-M4EifTMPNWMw7poclK&index=2)
-  [3강  Link](https://www.youtube.com/watch?v=36sjfxibME4&list=PL9mhQYIlKEheoq-M4EifTMPNWMw7poclK&index=3)

## GithubPages

-  [2강 Link](https://tykimos.github.io/2017/07/09/Training_Monitoring/)
-  [3강 Link](https://tykimos.github.io/2017/01/27/MLP_Layer_Talk/)
- [3강 실습 Link](https://tykimos.github.io/2017/02/04/MLP_Getting_Started/)

## 실습 dataset

- [Kaggle > 

```
# 코드로 형식 지정됨
```

피마족 인디언 당뇨병 발병 dataset](https://www.kaggle.com/kumargh/pimaindiansdiabetescsv)

## import libraries & dataset (mnist)

In [0]:
import keras
from keras import datasets
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation
import numpy as np

In [0]:
(X_train, y_train), (X_test, y_test) = datasets.mnist.load_data()

In [15]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

##  dataset 정규화 & One-Hot-Encoding (categorize)

###  dataset

- train data : 수능 모의고사
- test data  :  작년 수능시험  문제
- real data  :  올해 수능시험  문제

검증 dataset이 있어야 학습 중단 시점 결정 가능
-  overfitting 발생하는 구간 부근에서 학습 중단
-  스크린골프로 연습을 지나치게 하면,
 오히려 필드에서 못치게 된다

 ### channel of image

 - 흑백 :  channel 1개
 -  color : channel 3개   (R/G/B)
 - PNG : channel 4개 (R/G/B/a=투명)

In [0]:
X_train = X_train.reshape(-1, 28*28).astype('float32') / 255.0
X_test = X_test.reshape(-1, 28*28).astype('float32') / 255.0

y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

In [17]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 784), (60000, 10), (10000, 784), (10000, 10))

In [18]:
X_train[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [19]:
y_test[0]

array([0., 0., 0., 0., 0., 0., 0., 1., 0., 0.], dtype=float32)

###  검증 dataset  만들기

In [0]:
X_val = X_train[50000:]
y_val = y_train[50000:]

X_train = X_train[:50000]
y_train = y_train[:50000]

In [21]:
X_val.shape, y_val.shape, X_train.shape, y_train.shape

((10000, 784), (10000, 10), (50000, 784), (50000, 10))

## Make a Model

## Layers

### MLP
: Multi-Layer Perceptron

img example)

![https://tykimos.github.io/warehouse/2017-1-27_MLP_Layer_Talk_lego_3.png](https://tykimos.github.io/warehouse/2017-1-27_MLP_Layer_Talk_lego_3.png)

- inputs : 3
- weights : 3 * 2 = 6
- ouputs : 2

<br />

code example)
```python
Dense(8, input_dim=4, init='uniform', activation='relu'))
```

- `(units =) 8` : number of output neurons
- `input_dim = 4` : number of input neurons

여기서 학습해야할 data  수는 `units` * `input_dim` = 8 * 4 = 32

- `init=` : 가중치 초기화 방법
    - `'uniform'` : 균일 분포,  default
    - `'normal'` : gaussian 분포
- `activation=`: 활성화 함수, output을 어떻게 만들지
    - `linear`
        -  default
        -  input neurons  X 가중치  계산한 결과값 그대로 출력
        -  수치 예측하는  경우
    - `'relu'`
        -  rectifier function
        - 은닉층에서 주로 쓰임
    - `'Sigmoid'`
        -  0~1  사이  실수 출력
        -  binary classification 
    - `'Softmax'`
        - 각각의 확률을 더하면 100%
        - multi-classes classfication
- `output_dim=`
    - output 은닉층의 dimension
    - 마지막 은닉층의 output_dim은 y의 dim과 같아야 함
    -  y가 categorical type인 경우 category 수에 맞춰야

###  HyperParameter Tuning

- 바꿀 수 없는 option
    - 첫번째 layer의 input_dim  :  입력되는 data의 dimension에 맞춰야
    - sigmoid activation layer의 units : 0  또는 1이 출력되어야 하므로 반드시 1

In [0]:
model = Sequential()
model.add(Dense(units=64, input_dim = 28*28, activation='relu'))
model.add(Dense(units=10, activation='softmax'))

## Compile

- model의 학습과정 설정

### `caategorical_crossentropy`

- entropy : Youtube 이찬우 딥러닝 참고



In [0]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

## Fit

### `batch-size`

- 학습과정에서,  몇 문제를 풀고 network를 update할 것인지 결정

- memory와 학습 size  모두에 영향

- ex. 학습 데이터 =  100,  batch-size = 1  인 경우 : update  100회 발생

### `epoch`

- 같은 dataset을 몇 번 학습할 것인지??

- 같은 문제집이라도 반복하면 학습이 일어난다  - overfitting 가능성

### `validation_data=(X_val, y_val)`

- 한번의 epoch마다 학습한 내용을 가지고 검증

-  overfitting 일어나는 구간을 찾는 용도; val_accuracy가 감소하는 구간까지 epoch를 늘려서  찾아야 함


In [25]:
hist = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 3s 51us/step - loss: 0.1547 - accuracy: 0.9559 - val_loss: 0.1526 - val_accuracy: 0.9597
Epoch 2/20
50000/50000 [==============================] - 2s 48us/step - loss: 0.1487 - accuracy: 0.9576 - val_loss: 0.1485 - val_accuracy: 0.9600
Epoch 3/20
50000/50000 [==============================] - 3s 50us/step - loss: 0.1431 - accuracy: 0.9594 - val_loss: 0.1449 - val_accuracy: 0.9605
Epoch 4/20
50000/50000 [==============================] - 3s 61us/step - loss: 0.1382 - accuracy: 0.9603 - val_loss: 0.1401 - val_accuracy: 0.9628
Epoch 5/20
50000/50000 [==============================] - 3s 61us/step - loss: 0.1334 - accuracy: 0.9621 - val_loss: 0.1371 - val_accuracy: 0.9631
Epoch 6/20
50000/50000 [==============================] - 3s 61us/step - loss: 0.1289 - accuracy: 0.9635 - val_loss: 0.1349 - val_accuracy: 0.9637
Epoch 7/20
50000/50000 [==============================] - 2s 50us/st

# Pima Indian Diabetes

In [32]:
data = np.loadtxt('/content/pima-indians-diabetes.csv', delimiter=',')
data.shape

(768, 9)

In [33]:
data[:10]

array([[6.000e+00, 1.480e+02, 7.200e+01, 3.500e+01, 0.000e+00, 3.360e+01,
        6.270e-01, 5.000e+01, 1.000e+00],
       [1.000e+00, 8.500e+01, 6.600e+01, 2.900e+01, 0.000e+00, 2.660e+01,
        3.510e-01, 3.100e+01, 0.000e+00],
       [8.000e+00, 1.830e+02, 6.400e+01, 0.000e+00, 0.000e+00, 2.330e+01,
        6.720e-01, 3.200e+01, 1.000e+00],
       [1.000e+00, 8.900e+01, 6.600e+01, 2.300e+01, 9.400e+01, 2.810e+01,
        1.670e-01, 2.100e+01, 0.000e+00],
       [0.000e+00, 1.370e+02, 4.000e+01, 3.500e+01, 1.680e+02, 4.310e+01,
        2.288e+00, 3.300e+01, 1.000e+00],
       [5.000e+00, 1.160e+02, 7.400e+01, 0.000e+00, 0.000e+00, 2.560e+01,
        2.010e-01, 3.000e+01, 0.000e+00],
       [3.000e+00, 7.800e+01, 5.000e+01, 3.200e+01, 8.800e+01, 3.100e+01,
        2.480e-01, 2.600e+01, 1.000e+00],
       [1.000e+01, 1.150e+02, 0.000e+00, 0.000e+00, 0.000e+00, 3.530e+01,
        1.340e-01, 2.900e+01, 0.000e+00],
       [2.000e+00, 1.970e+02, 7.000e+01, 4.500e+01, 5.430e+02, 3.050e+01

In [34]:
type (data)

numpy.ndarray

In [39]:
!jupyter nbconvert --to markdown --stdout

This application is used to convert notebook files (*.ipynb) to various other
formats.


Options
-------

Arguments that take values are actually convenience aliases to full
Configurables, whose aliases are listed on the help line. For more information
on full configurables, see '--help-all'.

--execute
    Execute the notebook prior to export.
--allow-errors
    Continue notebook execution even if one of the cells throws an error and include the error message in the cell output (the default behaviour is to abort conversion). This flag is only relevant if '--execute' was specified, too.
--no-input
    Exclude input cells and output prompts from converted document. 
    This mode is ideal for generating code-free reports.
--stdout
    Write notebook output to stdout instead of files.
--stdin
    read a single notebook file from stdin. Write the resulting notebook with default basename 'notebook.*'
--inplace
    Run nbconvert in place, overwriting the existing notebook (only 
    relevan